In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-38301")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Lead Software Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyLead Software EngineerHilversum, North HollandBecome a Part of the NIKE, Inc. TeamNIKE, Inc. d

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Lead Software Engineer",
  "experience": "Proven experience in software development, with significant hands-on expertise in Java Spring Boot and Python",
  "skills": [
    "Java Spring Boot",
    "Python",
    "JavaScript frameworks (VueJS/React)",
    "AWS cloud services",
    "Microservices architecture",
    "API development",
    "Distributed systems",
    "DevOps tools (CI/CD pipelines, Jenkins, Git)",
    "Automated testing",
    "Agile methodologies"
  ],
  "description": "Lead the design, development and deployment of cloud-based services, primarily using Java Spring Boot and AWS. Architect, build and maintain services on AWS ECS, with Python utilized for Lambda functions. Guide the team in maintaining front-end services (VueJS), with long-term plans to transition to ReactJS. Collaborate with Architects, Product Managers and other engineering teams to deliver cutting-edge solutions that meet evolving consumer needs."
}
```


In [7]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead Software Engineer',
 'experience': 'Proven experience in software development, with significant hands-on expertise in Java Spring Boot and Python',
 'skills': ['Java Spring Boot',
  'Python',
  'JavaScript frameworks (VueJS/React)',
  'AWS cloud services',
  'Microservices architecture',
  'API development',
  'Distributed systems',
  'DevOps tools (CI/CD pipelines, Jenkins, Git)',
  'Automated testing',
  'Agile methodologies'],
 'description': 'Lead the design, development and deployment of cloud-based services, primarily using Java Spring Boot and AWS. Architect, build and maintain services on AWS ECS, with Python utilized for Lambda functions. Guide the team in maintaining front-end services (VueJS), with long-term plans to transition to ReactJS. Collaborate with Architects, Product Managers and other engineering teams to deliver cutting-edge solutions that meet evolving consumer needs.'}

In [8]:
type(json_res)

dict